In [1]:
from tensorflow.keras.models import Sequential
from tensorflow.keras import backend as K
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.utils import to_categorical
from keras.callbacks import EarlyStopping
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Input
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, Activation, MaxPooling2D, Flatten, Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import LeakyReLU


from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split

from imutils import paths
import matplotlib.pyplot as plt
import numpy as np
import argparse
import random
import cv2
import os

import warnings
from tqdm import tqdm_notebook as tqdm
import itertools
import tensorflow as tf

print("Tensorlfow Version: ", tf.__version__)
warnings.filterwarnings("ignore")
SEED = 42   # set random seed


Tensorlfow Version:  2.15.0


In [6]:
current_dir = os.getcwd()
trainpath = os.path.abspath(os.path.join(current_dir, '..', 'DATA', 'drowsiness','train'))

print(f"Train path: {trainpath}")


Train path: C:\Users\Harsh\Downloads\data_science01\dl_projects\task 3\DATA\drowsiness\train


In [7]:
## determine the total number of image paths in training, validation,
# and testing directories
totalTrain = len(list(paths.list_images(trainpath)))
# totalTest = len(list(paths.list_images(testpath)))
print(totalTrain)
# print(totalTest)

1452


In [5]:
# initialize the data and labels
print("[INFO] loading images...")
data = []
labels = []

# grab the image paths and randomly shuffle them
imagePaths = sorted(list(paths.list_images(trainpath)))
random.seed(42)
random.shuffle(imagePaths)

# progress bar 
with tqdm(total=len(imagePaths)) as pbar:
    
    # loop over the input images
    for idx, imagePath in enumerate(imagePaths):
        # load the image, pre-process it, and store it in the data list
        image = cv2.imread(imagePath)
        image = cv2.resize(image, (64, 64))
        image = img_to_array(image)
        data.append(image)

        # extract the class label from the image path and update the
        # labels list
        label = imagePath.split(os.path.sep)[-2]

        if label == "awake":
            label = 0
        elif label == "sleeping":
            label = 1
        # print("pr: ", label)	
        labels.append(label)
        
        # update the progressbar
        pbar.update(1)

[INFO] loading images...


0it [00:00, ?it/s]

In [5]:
labels

[1,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 0,
 0,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 1,
 1,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 1,
 0,
 1,
 1,
 0,
 1,
 1,
 0,
 1,
 0,
 1,
 1,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 1,
 1,
 1,
 0,
 0,
 1,
 0,
 1,
 0,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 0,
 0,
 1,
 0,
 1,
 1,
 1,
 0,
 0,
 1,
 1,
 0,
 1,
 0,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 0,
 1,
 0,
 0,
 1,
 1,
 1,
 0,
 1,
 0,
 1,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 1,
 1,
 0,
 0,
 1,
 1,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 1,
 0,
 0,
 0,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 1,
 1,
 0,
 1,
 0,
 1,
 0,
 1,


In [6]:
data[0]

array([[[ 93., 103., 143.],
        [ 97., 105., 145.],
        [ 97., 105., 146.],
        ...,
        [ 75.,  79.,  97.],
        [ 53.,  54.,  66.],
        [ 35.,  38.,  44.]],

       [[ 94., 104., 144.],
        [ 97., 105., 145.],
        [ 98., 106., 146.],
        ...,
        [ 81.,  85., 103.],
        [ 58.,  60.,  73.],
        [ 35.,  38.,  44.]],

       [[ 92., 102., 142.],
        [ 95., 103., 143.],
        [ 97., 105., 145.],
        ...,
        [ 85.,  89., 109.],
        [ 61.,  65.,  77.],
        [ 35.,  36.,  45.]],

       ...,

       [[109., 126., 165.],
        [112., 128., 167.],
        [114., 131., 170.],
        ...,
        [141., 150., 170.],
        [148., 158., 175.],
        [150., 159., 173.]],

       [[109., 126., 165.],
        [111., 128., 167.],
        [111., 129., 168.],
        ...,
        [139., 148., 169.],
        [148., 158., 175.],
        [149., 158., 172.]],

       [[109., 126., 166.],
        [109., 126., 166.],
        [108., 1

In [7]:
data = np.array(data, dtype="float") / 255.0
labels = np.array(labels)

In [8]:
(trainX, testX, trainY, testY) = train_test_split(data,labels, test_size=0.25, random_state=42)

In [9]:
trainX.shape

(1089, 64, 64, 3)

In [10]:
trainY.shape

(1089,)

In [11]:
trainY = to_categorical(trainY, num_classes=2)
testY = to_categorical(testY, num_classes=2)

In [12]:
# construct the image generator for data augmentation
aug = ImageDataGenerator(rotation_range=30, 
                         width_shift_range=0.1, 
                         height_shift_range=0.1, 
                         shear_range=0.2, 
                         zoom_range=0.2, 
                         horizontal_flip=True, 
                         fill_mode="nearest")

In [13]:
class LeNet:
    @staticmethod
    def build(width, height, depth, classes):
        # initialize the model
        model = Sequential()
        inputShape = (height, width, depth)  # (h, h, channel)

        # if we are using "channels first", update the input shape
        if K.image_data_format() == "channels_first":
            inputShape = (depth, height, width)

        # first set of CONV => LeakyReLU => POOL layers
        model.add(Conv2D(100, (5, 5), padding="same", input_shape=inputShape))
        model.add(LeakyReLU(alpha=0.1))
        model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))

        # second set of CONV => LeakyReLU => POOL layers
        model.add(Conv2D(200, (5, 5), padding="same"))
        model.add(LeakyReLU(alpha=0.1))
        model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))

        # third set of CONV => LeakyReLU => POOL layers
        model.add(Conv2D(400, (5, 5), padding="same"))
        model.add(LeakyReLU(alpha=0.1))
        model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))

        # first (and only) set of FC => LeakyReLU layers
        model.add(Flatten())
        model.add(Dense(800))
        model.add(LeakyReLU(alpha=0.1))

        # softmax classifier
        model.add(Dense(classes))
        model.add(Activation("sigmoid"))

        # return the constructed network architecture
        return model

In [14]:
EPOCHS = 50
INIT_LR = 1e-3
BS = 64

In [15]:
# initialize the model
print("[INFO] compiling model...")
model = LeNet.build(width=64, height=64, depth=3, classes=2)
opt = Adam(learning_rate=INIT_LR)

model.compile(loss="categorical_crossentropy", optimizer=opt, metrics=["accuracy"])

print("[INFO] model complied...")


[INFO] compiling model...


[INFO] model complied...


In [ ]:
# train the network
print("[INFO] training network...")
H = model.fit(x=aug.flow(trainX, trainY, batch_size=BS),
              validation_data=(testX, testY), 
              steps_per_epoch=len(trainX) // BS,
              epochs=EPOCHS, 
              verbose=1)


[INFO] training network...
Epoch 1/50


17/17 [==============================] - 21s 1s/step - loss: 2.7082 - accuracy: 0.4898 - val_loss: 0.7181 - val_accuracy: 0.4353
Epoch 2/50
17/17 [==============================] - 18s 1s/step - loss: 0.7215 - accuracy: 0.5073 - val_loss: 0.6898 - val_accuracy: 0.5124
Epoch 3/50
17/17 [==============================] - 18s 1s/step - loss: 0.6876 - accuracy: 0.5415 - val_loss: 0.7068 - val_accuracy: 0.5014
Epoch 4/50
17/17 [==============================] - 18s 1s/step - loss: 0.6767 - accuracy: 0.6498 - val_loss: 0.6341 - val_accuracy: 0.6529
Epoch 5/50
17/17 [==============================] - 17s 1s/step - loss: 0.6247 - accuracy: 0.6810 - val_loss: 0.5534 - val_accuracy: 0.7658
Epoch 6/50
17/17 [==============================] - 18s 1s/step - loss: 0.5376 - accuracy: 0.7580 - val_loss: 0.4216 - val_accuracy: 0.8292
Epoch 7/50
17/17 [==============================] - 18s 1s/step - loss: 0.5219 - accuracy: 0.7639 - val_loss: 0.3561 - val_accuracy

In [ ]:
model.save("driver_detection_50_epochs.model",save_format="h5")

In [ ]:
def plot_acc(H, N, plotPath=None): 

    plt.style.use("ggplot")
    plt.figure()
    plt.plot(np.arange(0, N), H.history["accuracy"], label="train_acc")
    plt.plot(np.arange(0, N), H.history["val_accuracy"], label="val_acc")
    plt.title("Training Accuracy")
    plt.xlabel("Epoch #")
    plt.ylabel("Accuracy")
    plt.legend(loc="lower left")
   

def plot_loss(H, N, plotPath=None):
    
    plt.style.use("ggplot")
    plt.figure()
    plt.plot(np.arange(0, N), H.history["loss"], label="train_loss")
    plt.plot(np.arange(0, N), H.history["val_loss"], label="val_loss")

    plt.title("Training Loss")
    plt.xlabel("Epoch #")
    plt.ylabel("Loss")
    plt.legend(loc="lower left")

In [ ]:
plot_acc(H,50)

In [ ]:
plot_loss(H,50)